# Global consumer electronic brand sales analysis

## Importing libraries and data

During this step, we import the necessary libraries and load the data required for further analysis. 


In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, ForeignKey
from sqlalchemy import Column, Integer, String, inspect, Float, Date, join, select
import os
import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode,iplot
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
init_notebook_mode(connected=True) 

In [4]:
engine = create_engine("sqlite:///database/my_database.db")
metadata = MetaData(bind=engine)

## Descriptive Statistics


This section offers a comprehensive overview of the dataset, including key descriptive statistics for the variables.


### Sales Dataset

In [6]:
sales_df = pd.read_sql('''SELECT * FROM sales;''', engine)

In [19]:
sales_df['Order Date'].min()

'2016-01-01 00:00:00.000000'

In [21]:
sales_df['Order Date'].max()

'2021-02-20 00:00:00.000000'

The dataset spans nearly five years, from January 1, 2016, to February 2021.

In [34]:
sales_df.value_counts('Currency Code', normalize=True).reset_index().rename(columns={0: 'Percentage'})

,Currency Code,Percentage
0,USD,0.536973
1,EUR,0.200703
2,GBP,0.129445
3,CAD,0.086111
4,AUD,0.046769


The company supports five currencies: USD, EUR, GBP, CAD, and AUD. The US dollar is the most commonly used, making up approximately 53% of all sales, while the Euro accounts for around 20%.

In [35]:
sales_df['Quantity'].describe()

count    62884.000000
mean         3.144790
std          2.256371
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max         10.000000
Name: Quantity, dtype: float64

The number of items per order ranges from 1 to 10, with an average purchase quantity of 3.

### Stores Dataset

In [38]:
stores_df = pd.read_sql('''SELECT * FROM stores;''', engine)

In [49]:
stores_df.value_counts('Country').reset_index().rename(columns={0: 'Count'})

,Country,Count
0,United States,24
1,Germany,9
2,France,7
3,United Kingdom,7
4,Australia,6
5,Canada,5
6,Netherlands,5
7,Italy,3


In [53]:
stores_df.value_counts('Country').reset_index().rename(columns={0: 'Count'})['Count'].sum()


66

In [55]:
stores_df.value_counts('Country').reset_index().rename(columns={0: 'Count'})['Count'].median()


6.5

The company operates 66 stores across 7 countries, with the United States having the highest count at 24 stores and Italy the lowest at 3 stores. The median number of stores per country is 6.

### Customers Dataset

In [ ]:
stores_df = pd.read_sql('''SELECT * FROM stores;''', engine)